In [1]:
import torch
if not torch.cuda.is_available():
  raise Exception("GPU not availalbe. CPU training will be too slow.")
print("device name", torch.cuda.get_device_name(0))

device name Tesla T4


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install soundfile
!pip install jiwer

In [ ]:
! git clone https://github.com/huggingface/transformers.git

In [5]:
! apt install git-lfs
! git config --global user.email "gary109@gmail.com"
! git config --global user.name "GARY"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [6]:
! huggingface-cli login
# from huggingface_hub import notebook_login
# notebook_login()


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machin

In [7]:
!git config --global credential.helper store

In [11]:
from datasets import load_dataset
dataset = load_dataset("orchid219.py", 'orchid219')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset crop14 downloaded and prepared to /root/.cache/huggingface/datasets/crop14/crop14-small/0.0.3/8b77fa03ec3a2deb8f9f785e107548d900945f9abf5ffa59799287a692ca326e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers.utils.dummy_vision_objects import ImageGPTFeatureExtractor
import random
from PIL import ImageDraw, ImageFont, Image

# def show_examples(ds, seed: int = 1234, examples_per_class: int = 3, size=(100, 100)):

#     w, h = size
#     labels = ds['train'].features['labels'].names
#     # labels = labels[:9]
#     grid = Image.new('RGB', size=(examples_per_class * w, len(labels) * h))
#     draw = ImageDraw.Draw(grid)
#     font = ImageFont.truetype("./fonts/LiberationMono-Bold.ttf", 24)
#     for label_id, label in enumerate(labels):

#         # Filter the dataset by a single label, shuffle it, and grab a few samples
#         ds_slice = ds['train'].filter(lambda ex: ex['labels'] == label_id).shuffle(seed).select(range(examples_per_class))

#         # Plot this label's examples along a row
#         for i, example in enumerate(ds_slice):
#             image = example['image']
#             idx = examples_per_class * label_id + i
#             box = (idx % examples_per_class * w, idx // examples_per_class * h)
#             grid.paste(image.resize(size), box=box)
#             draw.text(box, str(label), (255, 255, 255), font=font)

#     return grid

# show_examples(dataset, seed=random.randint(0, 1337), examples_per_class=3)
dataset['train'][0]['image']

In [14]:
dataset.push_to_hub("gary109/orchid219")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
! pip install accelerate deepspeed

In [36]:
! accelerate config

In which compute environment are you running? ([0] This machine, [1] AWS (Amazon SageMaker)): 0
Which type of machine are you using? ([0] No distributed training, [1] multi-CPU, [2] multi-GPU, [3] TPU): 0
Do you want to run your training on CPU only (even if a GPU is available)? [no]:
Do you want to use DeepSpeed? [yes/NO]: 
How many processes in total will you use? [1]: 
Do you wish to use FP16 or BF16 (mixed precision)? [NO/fp16/bf16]: fp16


In [33]:
! accelerate test


Running:  accelerate-launch --config_file=None /usr/local/lib/python3.7/dist-packages/accelerate/test_utils/test_script.py
stdout: [2022-04-26 03:32:13,390] [WARNING] [runner.py:155:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
stdout: [2022-04-26 03:32:13,398] [INFO] [runner.py:453:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 /usr/local/lib/python3.7/dist-packages/accelerate/test_utils/test_script.py
stdout: [2022-04-26 03:32:14,631] [INFO] [launch.py:96:main] 0 NCCL_VERSION=2.7.8
stdout: [2022-04-26 03:32:14,631] [INFO] [launch.py:103:main] WORLD INFO DICT: {'localhost': [0]}
stdout: [2022-04-26 03:32:14,631] [INFO] [launch.py:110:main] nnodes=1, num_local_procs=1, node_rank=0
stdout: [2022-04-26 03:32:14,631] [INFO] [launch.py:122:main] global_rank_mapping=defaultdict(<class 'list'>, {'localhost': [0]})
stdout: [2022-04-26 03:32:14,631] 

In [34]:
%cd /content/transformers/examples/pytorch/image-classification


/content/transformers/examples/pytorch/image-classification


- google/vit-base-patch16-224-in21k [NOW]
- google/vit-base-patch32-224-in21k
- google/vit-large-patch16-224-in21k
- google/vit-large-patch32-224-in21k

- google/vit-base-patch16-224
- google/vit-base-patch16-384
- google/vit-base-patch32-384

- google/vit-large-patch16-384
- google/vit-large-patch16-224
- google/vit-large-patch32-384

- google/vit-huge-patch14-224-in21k

In [ ]:
! accelerate launch run_image_classification.py \
    --dataset_name "gary109/orchid219" \
    --output_dir ./orchid219_vit-base-patch16-224-in21k/ \
    --remove_unused_columns False \
    --overwrite_output_dir \
    --do_train \
    --do_eval \
    --push_to_hub \
    --push_to_hub_model_id orchid219_vit-base-patch16-224-in21k \
    --learning_rate 2e-5 \
    --num_train_epochs 30 \
    --per_device_train_batch_size 80 \
    --per_device_eval_batch_size 16 \
    --logging_strategy steps \
    --logging_steps 10 \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --load_best_model_at_end True \
    --save_total_limit 3 \
    --seed 1337
    # --gradient_accumulation_steps 8 \
    # --gradient_checkpointing

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:978: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case gary109/vit-base-crop14-small).
  FutureWarning,
04/26/2022 03:42:21 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/26/2022 03:42:21 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStr